# Python Flight Mechanics Engine 

## Aircraft 

In order to perform a simulation, the first thing we need is an aircraft:

In [1]:
from pyfme.aircrafts import Cessna172

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
aircraft = Cessna172()

Aircraft will provide the simulator the forces, moments and inertial properties in order to perform the integration of the dynamic system equations:

In [4]:
print(f"Aircraft mass: {aircraft.mass} kg")
print(f"Aircraft inertia tensor: \n {aircraft.inertia} kg/m²")

Aircraft mass: 1043.2616 kg
Aircraft inertia tensor: 
 [[ 1285.3154166      0.             0.        ]
 [    0.          1824.9309607      0.        ]
 [    0.             0.          2666.89390765]] kg/m²


In [5]:
print(f"forces: {aircraft.total_forces} N")
print(f"moments: {aircraft.total_moments} N·m")

forces: [ 0.  0.  0.] N
moments: [ 0.  0.  0.] N·m


For the aircraft, in order to calculate its forces and moments it is necessary to set the controls values within the limits: 

In [6]:
print(aircraft.controls)

{'delta_elevator': 0, 'delta_aileron': 0, 'delta_rudder': 0, 'delta_t': 0}


In [7]:
print(aircraft.control_limits)

{'delta_elevator': (-0.4537856055185257, 0.48869219055841229), 'delta_aileron': (-0.26179938779914941, 0.3490658503988659), 'delta_rudder': (-0.27925268031909273, 0.27925268031909273), 'delta_t': (0, 1)}


but also to provide and environment (ie. atmosphere, winds, gravity) and the aircraft state, which will also determine the aerodynamic contribution.

## Environment 

In [8]:
from pyfme.environment.atmosphere import ISA1976
from pyfme.environment.wind import NoWind
from pyfme.environment.gravity import VerticalConstant

In [9]:
atmosphere = ISA1976()
gravity = VerticalConstant()
wind = NoWind()

The atmosphere, wind and gravity model make up the environment:

In [10]:
from pyfme.environment import Environment

In [11]:
environment = Environment(atmosphere, gravity, wind)

The environment has an update method which given the state (ie. position, altitude...) updates the environment variables (ie. density, wind magnitude, gravity force...)

In [12]:
help(environment.update)

Help on method update in module pyfme.environment.environment:

update(state) method of pyfme.environment.environment.Environment instance



## State 

Even if the state can be set manually by giving the position, attitude, velocity, angular velocities... Most of the times, the user will want to trim the aircraft in a stationary condition. The aircraft controls to flight in that condition will be also provided by the trimmer.

In [13]:
from pyfme.utils.trimmer import steady_state_trim

In [14]:
help(steady_state_trim)

Help on function steady_state_trim in module pyfme.utils.trimmer:

steady_state_trim(aircraft, environment, pos, psi, TAS, controls, gamma=0, turn_rate=0, exclude=None, verbose=0)
    Finds a combination of values of the state and control variables
    that correspond to a steady-state flight condition.
    
    Steady-state aircraft flight is defined as a condition in which all
    of the motion variables are constant or zero. That is, the linear and
    angular velocity components are constant (or zero), thus all
     acceleration components are zero.
    
    Parameters
    ----------
    aircraft : Aircraft
        Aircraft to be trimmed.
    environment : Environment
        Environment where the aircraft is trimmed including atmosphere,
        gravity and wind.
    pos : Position
        Initial position of the aircraft.
    psi : float, opt
        Initial yaw angle (rad).
    TAS : float
        True Air Speed (m/s).
    controls : dict
        Initial value guess for each con

In [15]:
from pyfme.models.state.position import EarthPosition

In [16]:
pos = EarthPosition(x=0, y=0, height=1000)
psi = 0.5  # rad
TAS = 45  # m/s
controls0 = {'delta_elevator': 0, 'delta_aileron': 0, 'delta_rudder': 0, 'delta_t': 0.5}

In [17]:
trimmed_state, trimmed_controls = steady_state_trim(
    aircraft,
    environment,
    pos,
    psi,
    TAS,
    controls0
)    

In [18]:
trimmed_controls

{'delta_aileron': 5.6949494207348974e-18,
 'delta_elevator': -0.048951124635247888,
 'delta_rudder': -1.4494655727415656e-17,
 'delta_t': 0.57799667845248459}

In [19]:
environment.update(trimmed_state)

Now, all the necessary elements in order to calculate forces and moments are available 

In [20]:
# Environment conditions for the current state:
environment.update(trimmed_state)

# Forces and moments calculation:
forces, moments = aircraft.calculate_forces_and_moments(trimmed_state, environment, controls0)

In [21]:
forces, moments

(array([  1.14823706e-11,  -6.00938052e-18,  -5.45696821e-12]),
 array([  1.34219095e-13,  -1.43613996e-11,  -2.41989038e-15]))

The aircraft is trimmed indeed: the total forces and moments (aerodynamics + gravity + thrust) are zero

## Simulation 

In order to simulate the dynamics of the aircraft under certain inputs in an environment, the user can set up a simulation using a dynamic system:

In [22]:
from pyfme.models import EulerFlatEarth

In [23]:
system = EulerFlatEarth(t0=0, full_state=trimmed_state)

### Constant Controls 

Let's set the controls for the aircraft during the simulation. As a first step we will set them constant and equal to the trimmed values.

In [24]:
from pyfme.utils.input_generator import Constant

In [25]:
controls = controls = {
    'delta_elevator': Constant(trimmed_controls['delta_elevator']),
    'delta_aileron': Constant(trimmed_controls['delta_aileron']),
    'delta_rudder': Constant(trimmed_controls['delta_rudder']),
    'delta_t': Constant(trimmed_controls['delta_t'])
}

In [26]:
from pyfme.simulator import Simulation

In [ ]:
sim = Simulation(aircraft, system, environment, controls)

Once the simulation is set, the propagation can be performed:

In [ ]:
results = sim.propagate(10)

The results are returned in a DataFrame:

In [ ]:
results

In [ ]:
%matplotlib inline

In [ ]:
kwargs = {'marker': '.',
          'subplots': True,
          'sharex': True,
          'figsize': (12, 6)}

In [ ]:
results.plot(y=['x_earth', 'y_earth', 'height'], **kwargs);

In [ ]:
results.plot(y=['psi', 'theta', 'phi'], **kwargs);

In [ ]:
results.plot(y=['v_north', 'v_east', 'v_down'], **kwargs);

In [ ]:
results.plot(y=['p', 'q', 'r'], **kwargs);

In [ ]:
results.plot(y=['alpha', 'beta', 'TAS'], **kwargs);

In [ ]:
results.plot(y=['Fx', 'Fy', 'Fz'], **kwargs);

In [ ]:
results.plot(y=['Mx', 'My', 'Mz'], **kwargs);

In [ ]:
results.plot(y=['elevator', 'aileron', 'rudder', 'thrust'], **kwargs);

### Doublet 

Let's set the controls for the aircraft during the simulation. As a first step we will set them constant and equal to the trimmed values.

In [ ]:
from pyfme.utils.input_generator import Doublet

In [ ]:
de0 = trimmed_controls['delta_elevator']

In [ ]:
controls = controls = {
    'delta_elevator': Doublet(t_init=2, T=1, A=0.1, offset=de0),
    'delta_aileron': Constant(trimmed_controls['delta_aileron']),
    'delta_rudder': Constant(trimmed_controls['delta_rudder']),
    'delta_t': Constant(trimmed_controls['delta_t'])
}

In [ ]:
sim = Simulation(aircraft, system, environment, controls)

Once the simulation is set, the propagation can be performed:

In [ ]:
results = sim.propagate(90)

In [ ]:
results.plot(y=['x_earth', 'y_earth', 'height'], **kwargs);

In [ ]:
results.plot(y=['psi', 'theta', 'phi'], **kwargs);

In [ ]:
results.plot(y=['v_north', 'v_east', 'v_down'], **kwargs);

In [ ]:
results.plot(y=['p', 'q', 'r'], **kwargs);

In [ ]:
results.plot(y=['alpha', 'beta', 'TAS'], **kwargs);

In [ ]:
results.plot(y=['Fx', 'Fy', 'Fz'], **kwargs);

In [ ]:
results.plot(y=['Mx', 'My', 'Mz'], **kwargs);

In [ ]:
results.plot(y=['elevator', 'aileron', 'rudder', 'thrust'], **kwargs);

## Propagating only one time step

In [ ]:
dt = 0.05  # seconds
sim = Simulation(aircraft, system, environment, controls, dt)

In [ ]:
results = sim.propagate(0.5)
results

We can propagate for one time step even once the simulation has been propagated before:

In [ ]:
results = sim.propagate(sim.time+dt)
results

Notice that `results` will include the previous timesteps as well as the last one. To get just the last one one can use pandas `loc` or `iloc`:

In [ ]:
results.iloc[-1]  # last time step

In [ ]:
results.loc[sim.time]  # results for current simulation time